# Recipe Rating Prediction

**Name(s)**: Andrew Kim, Kyle Le

**Website Link**: [(Recipe Rating Prediction)](https://kylele3221.github.io/recipe-rating-prediction/)

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from dsc80_utils import *

## Step 1: Introduction

### Understanding the Data

There are two data files, RAW_recipes.csv and interactions.csv, that will be referred to as the recipes and interactions data sets respectively.  Both data sets are scraped off of food.com.

## Step 2: Data Cleaning and Exploratory Data Analysis

In [ ]:
recipes = pd.read_csv("RAW_recipes.csv")
ratings = pd.read_csv("interactions.csv")


### Data Cleaning

In [ ]:
# Make a function to convert the nutrition column from a string into a list and apply it
def str_to_list(s):
    s = s.strip()[1:-1]
    if not s:
        return []
    return [float(item.strip()) for item in s.split(",")]

recipes["nutrition"] = recipes["nutrition"].apply(str_to_list)

# Split the nutrition column into each of its nutritional components
nutrition_columns = [
    "calories",
    "total fat",
    "sugar",
    "sodium",
    "protein",
    "saturated fat",
    "carbohydrates"
]
temp_df = pd.DataFrame(recipes["nutrition"].to_list(), columns=nutrition_columns)
recipes = recipes.drop(columns=["nutrition"])
recipes = pd.concat([recipes, temp_df], axis=1)

# Merge data frames
merged_df = recipes.merge(ratings, left_on='id', right_on='recipe_id', how='left')
df = merged_df

# Drop unncessary columns
df = df.drop(columns=['name', 'minutes', 'contributor_id', 'submitted', 'tags',
       'n_steps', 'steps', 'description', 'ingredients', 'n_ingredients',
       'user_id', 'recipe_id', 'date', 'review'])


# Get the average rating for each recipe
df = df.groupby('id').agg({
    'calories': 'first',
    'total fat': 'first',
    'sugar': 'first',
    'sodium': 'first',
    'protein': 'first',
    'saturated fat': 'first',
    'carbohydrates': 'first',
    'rating': 'mean'
}).reset_index()

df = df.rename(columns={'rating': 'average_rating'})

# 0 ratings to np.nan
df['average_rating'] = df['average_rating'].replace(0, np.nan)

df.head(5)

### Univariate Analysis

This section presents a univariate analysis to explore the relationship between product ratings and various nutrition facts, such as calories, fat, and protein content. The code summarizes the distribution of ratings across different levels of each nutritional variable, often visualizing these patterns with plots or summary statistics. By examining these relationships, we aim to identify any trends or associations, such as whether items with certain nutritional profiles tend to receive higher or lower ratings. This analysis helps to uncover potential factors that may influence how products are rated by consumers.


In [ ]:
# Univariate analysis for each rating.  Shows counts of eaching rating
rating_counts = df['average_rating'].value_counts().sort_index().reset_index()
rating_counts.columns = ['average_rating', 'count']

# Create a bar chart
fig = px.bar(
    rating_counts,
    x='average_rating', y='count',
    labels={'average_rating': 'Average Rating', 'count': 'Number of Ratings'},
    title='Distribution of Recipe Ratings'
)

fig.show()



# Univariate analysis for each nutritional component in histograms.  Shows counts of each component
nutrition_columns = [
    "calories", "total fat", "sugar", "sodium",
    "protein", "saturated fat", "carbohydrates"
]

# Set the maximum value for each nutritional component for the x-axis
x_max = {
    "calories": 2500,
    "total fat": 600,
    "sugar": 2000,
    "sodium": 2000,
    "protein": 300,
    "saturated fat": 500,
    "carbohydrates": 300
}

# Create histograms for each nutritional component
for col in nutrition_columns:
    fig = px.histogram(
        df,
        x=col,
        nbins=50,
        title=f'Distribution of {col.title()}',
        labels={col: col.title()},

        # To help with showing most of the data, hides outliers
        range_x=[0, x_max[col]]
    )
    fig.show()

### Bivariate Analysis

In this section, we conduct a bivariate analysis to investigate how product ratings relate to specific nutrition facts, such as calories, fat, or protein. The code examines the association between ratings and each nutritional variable by comparing the distribution of ratings across different levels or categories of the nutrition fact. Visualization techniques (such as scatter plots or boxplots) and summary statistics are used to explore whether there is a systematic relationship such as higher ratings for lower-calorie items or vice versa. This analysis provides deeper insight into how multiple variables interact and can reveal potential patterns that might not be evident in univariate analyses.  Plotting all of the data points caused extreme lag so a random set of 5000 data points were chosen to represent the data.


In [ ]:
nutrients = [
    "calories",
    "total fat",
    "sugar",
    "sodium",
    "protein",
    "saturated fat",
    "carbohydrates"
]


np.random.seed(42)

df_sample = df.sample(n=5000, random_state=42) if len(df) > 5000 else df.copy()

# Using graphs to visualize relationships
for nutrient in nutrients:
    # Plot the downsampled data
    plt.scatter(df_sample['average_rating'], df_sample[nutrient],  alpha=0.2)
    plt.xlabel('Rating')
    plt.ylabel('Calories')
    plt.title('Scatter plot of Calories vs Rating (Downsampled)')
    plt.show()

### Interesting Aggregates

This section explores interesting aggregate statistics derived from the dataset, focusing on summary measures that reveal broader patterns or notable outliers. The code calculates group level metrics, such as the average rating within categories defined by nutrition facts. By aggregating the data in different ways, we can identify standout items, observe trends across groups, and highlight key insights that might be obscured at the individual level. These aggregates help to summarize the data and point to potential areas for further investigation.


In [ ]:
# Creates bins of uniform width 500
bin_width = 500
max_cut = 2500

cal_edges = np.arange(0, max_cut + bin_width, bin_width)

# Cuts off any values above 2500
df["calories_clipped"] = df["calories"].clip(upper=max_cut)

# Equal bins
df["calories_bin"] = pd.cut(
    df["calories_clipped"],
    bins=cal_edges,
    include_lowest=True
)

# Compute mean rating per bin
agg_cal = (
    df
    .groupby("calories_bin", observed=True)["average_rating"]
    .mean()
    .reset_index()
    .dropna()
)

agg_cal["bin_str"] = agg_cal["calories_bin"] \
    .apply(lambda iv: f"{int(iv.left)}–{int(iv.right)}")

fig = px.bar(
    agg_cal,
    x="bin_str",
    y="average_rating",
    title="Average Recipe Rating by Calories (Uniform 500 Cal Bins)",
    labels={"bin_str": "Calories Range", "average_rating": "Average Rating"}
)
fig.update_layout(
    xaxis={
        "categoryorder":"array",
        "categoryarray": agg_cal["bin_str"].tolist()
    }
)
fig.show()



## Step 3: Assessment of Missingness

### NMAR Analysis

In [ ]:
# Isolating NaN value rows
nan_rows = df[df.isna().any(axis=1)]
print(nan_rows)

In the dataset, all missing values occur in the average_rating column. Any 0s in this column with NaN values, under the assumption that a rating of 0 did not represent a real rating but rather a missing or unreported value. This means that the missingness in the average_rating column is directly related to the value itself specifically, the missingness occurs when the original value was 0.

Since the probability of a value being missing is determined by its value, the missingness mechanism is NMAR. We cannot determine this just by analyzing the observed data. Instead, it requires understanding the data generating process and recognizing that missing ratings are systematically related to their values.



### Missingness Dependency

In [ ]:
merged_df['has_rating'] = merged_df['rating'].notna().astype(int)
merged_df['missing_review'] = merged_df['review'].isna()

# Permutation test function
def permutation_test(df, test_col, missing_indicator, n_permutations=1000):
    temp_df = df.dropna(subset=[test_col])
    one = temp_df[temp_df[missing_indicator]][test_col]
    zero = temp_df[~temp_df[missing_indicator]][test_col]
    observed_diff = one.mean() - zero.mean()
    differences = []
    for i in range(n_permutations):
        shuffled = np.random.permutation(temp_df[missing_indicator])
        difference = temp_df[test_col][shuffled].mean() - temp_df[test_col][~shuffled].mean()
        differences.append(difference)
    p_value = np.mean(np.abs(differences) >= np.abs(observed_diff))
    return observed_diff, p_value, np.array(differences)

obs_diff_rating, p_val_rating, differences_rating = permutation_test(merged_df, 'has_rating', 'missing_review')

fig_perm = go.Figure()
fig_perm.add_trace(go.Histogram(
    x=differences_rating,
    nbinsx=30,
    name="Permutation Differences",
    opacity=0.7
))
fig_perm.add_vline(
    x=obs_diff_rating,
    line_dash="dash",
    line_color="red",
    annotation_text="Observed Diff",
    annotation_position="top right"
)
fig_perm.update_layout(
    title="Permutation Test Distribution for Rating Missingness",
    xaxis_title="Difference in Mean (has_rating: review missing - review present)",
    yaxis_title="Count"
)
fig_perm.show()

temp_df = merged_df.dropna(subset=['minutes'])
fig_box = px.box(
    temp_df,
    x='missing_review',
    y='minutes',
    labels={'missing_review': 'Review Missing', 'minutes': 'Minutes'},
    title='Distribution of Minutes by Review Missingness'
)
fig_box.show()


Permutation tests were conducted to analyze the dependency of missingness in the review column on both the presence of a rating and the preparation time. The observed difference in the probability of having a rating between rows with and without missing reviews was -0.02, with a p-value of 0.0000, indicating a statistically significant association. In contrast, the observed difference in the average preparation time was 33.56 minutes, with a p-value of 0.6610, suggesting no statistically significant association. These results provide evidence that the missingness in the review column is related to the presence of a rating, but not to the preparation time of the recipe.

## Step 4: Hypothesis Testing

Null hypothesis (H₀): The average rating of low-calorie recipes is equal to the average rating of high-calorie recipes.

Alternative hypothesis (H₁): The average rating of low-calorie recipes is not equal to the average rating of high-calorie recipes

In [ ]:
# Split the data into two groups based on calorie level and extract the average ratings
low_cal = df[df['calories'] <= 500]['average_rating'].dropna()
high_cal = df[df['calories'] > 500]['average_rating'].dropna()

# Compute the observed difference in average rating between high- and low-calorie recipes
observed_diff = high_cal.mean() - low_cal.mean()
print("Observed difference:", observed_diff)

# Combine the two groups into one dataset
combined = pd.concat([low_cal, high_cal])
labels = ['low'] * len(low_cal) + ['high'] * len(high_cal)

n_reps = 1000
diffs = []

# Run permutation test
for _ in range(n_reps):
    shuffled = np.random.permutation(labels)
    group1 = combined[np.array(shuffled) == 'low']
    group2 = combined[np.array(shuffled) == 'high']
    diffs.append(group2.mean() - group1.mean())

# Compute the p-value: proportion of permuted diffs as or more extreme than the observed
p_val = np.mean(np.abs(diffs) >= np.abs(observed_diff))
print("p-value:", p_val)

## Step 5: Framing a Prediction Problem

Prediction: Can we predict the average rating of a recipe based on its nutritional components? For the purposes of this project, we will use calories and sugar.

Justification: This is a realistic prediction task because nutrition facts are known at the time a recipe is published, but user ratings are not. Predicting ratings could help surface highly rated recipes early on.

## Step 6: Baseline Model

In [ ]:
df_clean = df[['calories', 'sugar', 'average_rating']].dropna()
X = df_clean[['calories', 'sugar']]
y = df_clean['average_rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
baseline_model = LinearRegression()
baseline_model.fit(X_train, y_train)
y_pred = baseline_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Baseline Model Mean Squared Error: {mse:.4f}')
print('Intercept:', baseline_model.intercept_)
print('Coefficients:', baseline_model.coef_)

plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--')
plt.xlabel('Actual Average Rating')
plt.ylabel('Predicted Average Rating')
plt.title('Baseline Model: Predicted vs. Actual Ratings')
plt.tight_layout()
plt.show()


## Step 7: Final Model

In [ ]:
# Add engineered features to the train and test sets
X_train_f = X_train.copy()
X_test_f = X_test.copy()

# Feature 1: Interaction between calories and sugar
X_train_f['calories_x_sugar'] = X_train_f['calories'] * X_train_f['sugar']
X_test_f['calories_x_sugar'] = X_test_f['calories'] * X_test_f['sugar']

# Feature 2: Log-transformed calories
X_train_f['log_calories'] = np.log1p(X_train_f['calories'])
X_test_f['log_calories'] = np.log1p(X_test_f['calories'])

# Fit a Ridge Regression by hand and grid search alpha
feature_cols = ['calories', 'sugar', 'calories_x_sugar', 'log_calories']

# Standardize features based on training data
means = X_train_f[feature_cols].mean()
stds = X_train_f[feature_cols].std()
X_train_scaled = (X_train_f[feature_cols] - means) / stds
X_test_scaled = (X_test_f[feature_cols] - means) / stds

# Manual grid search for Ridge penalty
alphas = [0.01, 0.1, 1, 10, 100]
best_alpha = None
best_mse = np.inf
best_coefs = None
best_intercept = None

for i in alphas:
    # Closed-form Ridge regression: w = (X^T X + alpha*I)^-1 X^T y
    X_mat = np.hstack([np.ones((X_train_scaled.shape[0], 1)), X_train_scaled.values])
    I = np.eye(X_mat.shape[1])
    I[0, 0] = 0  # Don't regularize intercept
    w = np.linalg.inv(X_mat.T @ X_mat + i * I) @ (X_mat.T @ y_train.values)
    X_test_mat = np.hstack([np.ones((X_test_scaled.shape[0], 1)), X_test_scaled.values])
    y_pred_final = X_test_mat @ w
    mse = ((y_pred_final - y_test.values) ** 2).mean()
    if mse < best_mse:
        best_mse = mse
        best_alpha = i
        best_coefs = w[1:]
        best_intercept = w[0]

print(f'Final Model Mean Squared Error: {best_mse:.4f}')
print(f'Best alpha (ridge penalty): {best_alpha}')
print('Intercept:', best_intercept)
print('Coefficients:', best_coefs)


## Step 8: Fairness Analysis

In [ ]:
# 1) Reconstruct the "calories" array for each test-row so we can form the two groups
cal_test = X_test_f['calories']
cal_test

# 2) Boolean masks on test set
cutoff = 500
is_low = cal_test <= cutoff
is_high = cal_test > cutoff

# 3) Extract true and predicted ratings for each group
mask_low = is_low.values
mask_high = is_high.values

y_true_low = y_test.values[mask_low]
y_true_high = y_test.values[mask_high]

y_pred_low = y_pred_final[mask_low]
y_pred_high = y_pred_final[mask_high]

# Observed RMSE Difference
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_pred - y_true) ** 2))

rmse_low = rmse(y_true_low, y_pred_low)
rmse_high = rmse(y_true_high, y_pred_high)

observed_diff = rmse_high - rmse_low
observed_diff

<h3>Hypotheses for Fairness Analysis</h3>
- Null Hypothesis (H0): RMSE(high) - RMSE(low) = 0

- Alternative Hypothesis (H1): RMSE(high) - RMSE(low) > 0 
    (could do double sided but the theme of this project has been that higher calories  may be worse)

In [ ]:
# Pool of Test-set Errors and Group Labels

# 1) Compute squared errors for each test example
errors = (y_pred_final - y_test.values) ** 2

# 2) Create a label array: "L" for low-cal, "H" for high-cal
labels = np.where(is_low, "L", "H")

print("Number of low‐cal test rows: ", np.sum(labels == "L"))
print("Number of high‐cal test rows:", np.sum(labels == "H"))

In [ ]:
# Permutation test

n_reps = 1000
perm_diffs = np.empty(n_reps)

n_low = np.sum(labels == "L")
n_high = np.sum(labels == "H")

for i in range(n_reps):
    permuted_labels = np.random.permutation(labels)
    errors_L = errors[permuted_labels == "L"]
    errors_H = errors[permuted_labels == "H"]
    
    rmse_L = np.sqrt(np.mean(errors_L))
    rmse_H = np.sqrt(np.mean(errors_H))
    
    perm_diffs[i] = rmse_H - rmse_L

p_val = np.mean(np.abs(perm_diffs) >= np.abs(observed_diff))
print(f"Permutation test p-value: {p_val:.4f}")


<h3>
    Fairness Analysis Interpretation
</h3>

- Because our p value was 0.6220 which is greater than 0.05, we fail to reject the null. We did not find evidence of a difference. The model's error on high-calories vs. low-calorie recipes could plausibly be the same, up to random variation.